In [1]:
import urllib2
import time
import pdb
import xlwt
import requests, re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
%pdb on

Automatic pdb calling has been turned ON


In [8]:
# read url from file (length should be 38594)
with open('hb_url_pool_laptop.txt') as f:
    url_pool = f.readlines()
print len(url_pool)
print url_pool[584]


9591
http://www.std.gov.cn/hb/search/stdHBDetailed?id=66A17FF8BA3053BFE05397BE0A0AB67B



In [9]:
output = xlwt.Workbook(encoding = 'utf-8')
table = output.add_sheet('sheet1', cell_overwrite_ok = False)
row_idx = 0

In [ ]:
# for each page, find all the urls
# url_pool = ['http://www.std.gov.cn/db/search/stdDBDetailed?id=5DDA8BA3779B18DEE05397BE0A0A95A7']
idx = -1
for url in url_pool:
    # find all the urls    
    idx += 1
    while True:
        time.sleep(1)
        if idx %100 == 0:
            print idx
        try:
            response = urllib2.urlopen(url)  
        except urllib2.HTTPError, e:
            print str(e.reason)
            continue
        break
    html = response.read()
#     print html
    
#     pdb.set_trace()
    # initialize items to save
    tag_sp, sp = [],[]
    [tc_name, tc_number,tc_pubdate, tc_impdate, stop_date, tc_chistd, tc_interstd, tc_charge, tc_guikou, tc_type, draft_unit, drafter, backup_num, backup_file] = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
  
    # check if the page exists
    if "行业标准" not in html:
        print 'The request page does not exist'
        print url
        html = []
        continue
    
    sp = html.split("\n")
    len_sp = len(sp)
#     print sp
    for i, info in enumerate(sp):
        if i <=200 or i>=500:
            continue
#         print i,info
        # name
        if not tc_name and "container main-body" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 5].split(',sans-serif;">')
            tc_name = tag_sp[-1].strip('</h4>\r')
#             print idx, tc_name
            continue
            
        # standard number
        if not tc_number and "标准号" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_number = tag_sp[-1].strip("</dd>\r")
#             print tc_number
            continue
            
        if not tc_pubdate and "发布日期" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_pubdate = tag_sp[-1].strip("</dd>\r")
#             print tc_pubdate
            continue
        
        if not tc_impdate and "实施日期" in info:
#             tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
#             pdb.set_trace()
            tc_impdate = sp[i+2]
#             print tc_impdate
            continue
            
        if not tc_chistd and "中国标准分类号" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_chistd = tag_sp[-1].strip("</dd>\r")
#             print tc_chistd
            continue        
        
        if not tc_interstd and "国际标准分类号" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 2].split('</div> " >')
            tc_interstd = tag_sp[-1].strip('</span>\r')
#             tc_interstd = []
#             for item in tag_sp:
#                 if "</div>" in item:
#                     tc_interstd.append(item.strip("</div>")[0]) 
#             print tc_interstd
            continue

        if not tc_charge and '<dt class="basicInfo-item name">主管部门</dt>' in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value"><a class="triggers" href="" CD_CODE="1" target="_blank">')
            tc_charge = tag_sp[-1].strip("</a></dd>\r")
#             print tc_charge
            continue   
    
        if not tc_guikou and '<dt class="basicInfo-item name">归口单位</dt>' in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('target="_blank">')
            tc_guikou = tag_sp[-1].strip("</a></dd>\r")
#             print tc_charge
            continue  

        if not tc_type and "行业分类" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 1].split('<dd class="basicInfo-item value">')
            tc_type = tag_sp[-1].strip("</dd>\r")
#             print tc_type
            continue    
            
        if not stop_date and "标准状态" in info:
#             pdb.set_trace()
            tag_sp = sp[i + 8].split('">废止')
            stop_date =  tag_sp[-1].strip("</span></a></li>\r")
#             print stop_date
            continue

       
        if not draft_unit and '<h2 class="title-text">起草单位</h2>' in info:
#             pdb.set_trace()
            draft_unit = []
            idx2 = 1
            while '<dd class="basicInfo-item value">' not in sp[i+idx2] and idx2 + i < len_sp:
                idx2 += 1
            while '</div>' not in sp[i+idx2] and idx2 + i < len_sp:
                if '<dd class="basicInfo-item value">' in sp[i+idx2]:
                    tag_sp = sp[i + idx2].split('</span>')
                    draft_unit.append(tag_sp[-1].strip("</a></dd>\r")+'\r')
                idx2 += 1
#             print draft_unit
            continue

        if not drafter and '<h2 class="title-text">起草人</h2>' in info:
#             pdb.set_trace()
            drafter = []
            idx2 = 1
            while '<dd class="basicInfo-item twod"' not in sp[i+idx2] and idx2 + i < len_sp:
                idx2 += 1
            while '</div>' not in sp[i+idx2] and idx2 + i < len_sp:
                if '<dd class="basicInfo-item twod"' in sp[i+idx2]:
                    tag_sp = sp[i + idx2].split('</span>')
                    drafter.append(tag_sp[-1].strip("</dd>\r")+'\r')
                idx2 += 1
#             print drafter
            continue
 
        if not backup_num and "备案号" in info:
#             pdb.set_trace()
            tag_sp = sp[i].split('备案号：')
            backup_num = tag_sp[-1].strip("。</p>\r")
#             print backup_num
            continue
            
        if not backup_file and "备案公告" in info:
#             pdb.set_trace()
            tag_sp = sp[i+3].split("target='_blank'>")
            backup_file = tag_sp[-1].strip("</a>\r")
#             print backup_file
            continue    
            
    
    output_tbl = [tc_name, tc_number,tc_pubdate, tc_impdate, stop_date, tc_chistd, tc_interstd, tc_charge, tc_guikou, tc_type, draft_unit, drafter, backup_num, backup_file]
#     pdb.set_trace
#     print output_tbl
    if tc_number:
        # encode unit-8 (chinese)
        [table.write(row_idx, i, output_tbl[i]) for i in range(len(output_tbl))]
        row_idx += 1
        tc_number = []
#         print >> output, "\t".join(output_tbl)
    if idx % 100 == 0:
        output.save("/Volumes/data1/yefei_hb.xls")


0
100
200
300
400
500
600
